In [13]:
#score: 0.98093
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools
import pandas as pd

In [14]:
# import mnist data and visualize first image
X_train = np.load("/kaggle/input/audio-binary-classification/train_data.npy")
X_test = np.load("/kaggle/input/audio-binary-classification/test_data.npy")
y_train = pd.read_csv('/kaggle/input/audio-binary-classification/train_labels.csv').loc[:,'Label'].values

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train = np.reshape(X_train, (-1, 441,100,1))
X_test = np.reshape(X_test, (X_test.shape[0], 441,100,1))


In [15]:
print('Build LSTM RNN model ...')
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2],1)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(units=64, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
#model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

model.summary()

Build LSTM RNN model ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 437, 96, 64)       1664      
_________________________________________________________________
dropout_9 (Dropout)          (None, 437, 96, 64)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 433, 92, 32)       51232     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 216, 46, 32)       0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 216, 1472)         0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                393472    
_________________________________________________________________
dense_4 (Dense)              (None, 1)             

In [16]:

# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 200, epochs = 100)

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = model.predict(X_test)
#np.savetxt("y_pred_ann.csv", y_pred, delimiter=",")
  
df2 = pd.DataFrame(y_pred, index = range(y_pred.shape[0]))
df2.reset_index(level=0, inplace=True)
df2.columns=['Id', 'Label']
df2.to_csv("y_pred_conv_LSTM_64.csv", index=False)


Epoch 1/50
3999/3999 [==============================] - 36s 9ms/step - loss: 0.2586 - acc: 0.8657
Epoch 2/50
3999/3999 [==============================] - 24s 6ms/step - loss: 0.2306 - acc: 0.8780
Epoch 3/50
3999/3999 [==============================] - 24s 6ms/step - loss: 0.2263 - acc: 0.8785
Epoch 4/50
3999/3999 [==============================] - 24s 6ms/step - loss: 0.2219 - acc: 0.8797
Epoch 5/50
3999/3999 [==============================] - 24s 6ms/step - loss: 0.2194 - acc: 0.8830
Epoch 6/50
3999/3999 [==============================] - 24s 6ms/step - loss: 0.2195 - acc: 0.8867
Epoch 7/50
3999/3999 [==============================] - 24s 6ms/step - loss: 0.2226 - acc: 0.8777
Epoch 8/50
3999/3999 [==============================] - 24s 6ms/step - loss: 0.2121 - acc: 0.8892
Epoch 9/50
3999/3999 [==============================] - 24s 6ms/step - loss: 0.2085 - acc: 0.8967
Epoch 10/50
3999/3999 [==============================] - 24s 6ms/step - loss: 0.2026 - acc: 0.8970
Epoch 11/50
3999/39